In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Valoración y griegas de:
  • Call vanilla FX (Black-Scholes)
  • Call asiática aritmética (Monte Carlo)
Usa curvas de tasas doméstica y foránea e
implementa diferenciación automática con PyTorch.
"""
import torch
from torch.distributions.normal import Normal

torch.set_default_dtype(torch.float64)          # Mejor precisión para finanzas


# —­­ Parámetros base —
S0     = torch.tensor(800.0, requires_grad=True)  # spot
K      = 800.0
sigma  = torch.tensor(0.20, requires_grad=True)   # volatilidad
T      = torch.tensor(1.0166666)                                  # 1 mes (~0.0822 años)
r_f = torch.tensor(0.03, requires_grad=True)  # tasa foránea
r_d = torch.tensor(0.02, requires_grad=True)  # tasa doméstica
dW = torch.randn(100_000)
s = S0*torch.exp((r_f - r_d - 0.5*sigma**2) * T + sigma * torch.sqrt(T) * dW)
payoff = torch.maximum(s-K, torch.tensor(0.0))   # Payoff del call vanilla
price = (payoff * torch.exp(-r_d*T) ).mean()  # Valor esperado del call vanilla

def greeks(price, params):
    """
    Calcula las griegas de un call vanilla FX usando diferenciación automática.
    :param price: Precio del call.
    :param params: Lista de parámetros [S0, sigma, z_d, z_f].
    :return: Delta, Vega, Rho_doméstico, Rho_foráneo.
    """
    S0, sigma, z_d, z_f = params
    dS = torch.autograd.grad(price, S0, retain_graph=True)[0]
    dvol = torch.autograd.grad(price, sigma, retain_graph=True)[0]
    dzd = torch.autograd.grad(price, z_d, retain_graph=True)[0]
    dzf = torch.autograd.grad(price, z_f, retain_graph=True)[0]
    
    return dS, dvol, dzd, dzf

dS_a, dvol, dzd_a, dzf_a = greeks(price, [S0, sigma, r_d, r_f])


# —­­ Resultados —
print("=== Call VANILLA FX ===")
print(f"Precio: {price.item():.2f}")
print(f"Delta: {dS_a.item():.4f}")
print(f"Vega: {dvol.item():.4f}")
print(f"Rho doméstico: {dzd_a.item():.4f}")
print(f"Rho foráneo: {dzf_a.item():.4f}")

744.85780214430570000000
744.85780214430570000000
744.85780214430570000000


744.85780214430440000000

=== Call VANILLA FX ===
Precio: 67.58
Delta: 0.5547
Vega: 316.2263
Rho doméstico: -519.8755
Rho foráneo: 451.1729


In [18]:
torch.exp(-r_d*T)

tensor(0.9799, grad_fn=<ExpBackward0>)